<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#The-end." data-toc-modified-id="The-end.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>The end.</a></span></li></ul></div>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [22]:
# download of the data from url address and creating BeautifulSoup
# object taking only the content of html document
url = 'https://www.forexfactory.com/calendar.php?day=today'
page = requests.get(url)
content = page.content

soup = BeautifulSoup(content, 'html.parser')
#soup.prettify()

In [26]:
# scraping part of the code and locating data into forcal list of dictionaries
table = soup.find_all('tr',{'class':'calendar_row'})
#print(table)

forcal = []
for item in table:
    dict = {}
    dict['Currency'] = item.find_all('td', {'class':'calendar__currency'})[0].text.strip()
    dict['Event'] = item.find_all('td', {'class':'calendar__event'})[0].text.strip()
    dict['Time_Eastern'] = item.find_all('td',{'class':'calendar__time'})[0].text
    impact = item.find_all('td',{'class':'impact'})
    
    for icon in range(0, len(impact)):
        dict['Impact'] = impact[icon].find_all('span')[0]['title'].split(' ', 1)[0]
        
    dict['Actual'] = item.find_all('td', {'class':'calendar__actual'})[0].text
    dict['Forecast'] = item.find_all('td',{'class':'calendar__forecast'})[0].text
    dict['Previous'] = item.find_all('td', {'class':'calendar__previous'})[0].text

    forcal.append(dict)
    

In [33]:
# forcal list of dictionaries content
forcal[:2]

[{'Actual': '-0.2%',
  'Currency': 'JPY',
  'Event': 'All Industries Activity m/m',
  'Forecast': '-0.3%',
  'Impact': 'Low',
  'Previous': '-0.6%',
  'Time_Eastern': '12:30am'},
 {'Actual': '',
  'Currency': 'USD',
  'Event': 'FOMC Member Evans Speaks',
  'Forecast': '',
  'Impact': 'Medium',
  'Previous': '',
  'Time_Eastern': '2:00am'}]

In [31]:
# converting scraped data into pandas DataFrame
df = pd.DataFrame(forcal)
df = df[['Currency', 'Event', 'Impact', 'Time_Eastern', 'Actual', 'Forecast', 'Previous']]

In [32]:
# saving to html format file
df.to_html('forexcal.html')